In [1]:
import pandas as pd
import numpy as np
import re
import os

In [2]:
# reading Bible from file

lines = []
file = open("kjv-1769-clean.txt")
lines.append(file.read())

In [3]:
lines_split = lines[0].split("\n")

In [4]:
lines_split[:5]

['"\'\'"',
 '"In the beginning God created the heaven and the earth."',
 '"And the earth was without form, and void; and darkness was upon the face of the deep. And the Spirit of God moved upon the face of the waters."',
 '"And God said, Let there be light: and there was light."',
 '"And God saw the light, that it was good: and God divided the light from the darkness."']

In [5]:
lines = pd.DataFrame(lines_split, columns=['lines'])

In [6]:
lines.head()

,lines
0,"""''"""
1,"""In the beginning God created the heaven and t..."
2,"""And the earth was without form, and void; and..."
3,"""And God said, Let there be light: and there w..."
4,"""And God saw the light, that it was good: and ..."


In [7]:
lines = lines[lines.lines != '"\'\'"']

In [8]:
lines.head()

,lines
1,"""In the beginning God created the heaven and t..."
2,"""And the earth was without form, and void; and..."
3,"""And God said, Let there be light: and there w..."
4,"""And God saw the light, that it was good: and ..."
5,"""And God called the light Day, and the darknes..."


In [9]:
def clean_text(sentence):
    sentence = sentence.lower()

    sentence = re.sub(r"i'm", "i am", sentence)
    sentence = re.sub(r"i’m", "i am", sentence)

    sentence = re.sub(r"he's", "he is", sentence)
    sentence = re.sub(r"he’s", "he is", sentence)

    sentence = re.sub(r"she's", "she is", sentence)
    sentence = re.sub(r"she’s", "she is", sentence)

    sentence = re.sub(r"it's", "it is", sentence)
    sentence = re.sub(r"it’s", "it is", sentence)

    sentence = re.sub(r"that's", "that is", sentence)
    sentence = re.sub(r"that’s", "that is", sentence)

    sentence = re.sub(r"what's", "what is", sentence)
    sentence = re.sub(r"what’s", "what is", sentence)

    sentence = re.sub(r"where's", "where is", sentence)
    sentence = re.sub(r"where’s", "where is", sentence)

    sentence = re.sub(r"there's", "there is", sentence)
    sentence = re.sub(r"there’s", "there is", sentence)

    sentence = re.sub(r"who's", "who is", sentence)
    sentence = re.sub(r"who’s", "who is", sentence)

    sentence = re.sub(r"how's", "how is", sentence)
    sentence = re.sub(r"how’s", "how is", sentence)

    sentence = re.sub(r"\'ll", " will", sentence)
    sentence = re.sub(r"’ll", " will", sentence)

    sentence = re.sub(r"\'ve", " have", sentence)
    sentence = re.sub(r"’ve", " have", sentence)

    sentence = re.sub(r"\'re", " are", sentence)
    sentence = re.sub(r"’re", " are", sentence)

    sentence = re.sub(r"\'d", " would", sentence)
    sentence = re.sub(r"’d", " would", sentence)

    sentence = re.sub(r"won't", "will not", sentence)
    sentence = re.sub(r"won’t", "will not", sentence)

    sentence = re.sub(r"can't", "cannot", sentence)
    sentence = re.sub(r"can’t", "cannot", sentence)

    sentence = re.sub(r"n't", " not", sentence)
    sentence = re.sub(r"n’t", " not", sentence)

    sentence = re.sub(r"n'", "ng", sentence)
    sentence = re.sub(r"n’", "ng", sentence)

    sentence = re.sub(r"'bout", "about", sentence)
    sentence = re.sub(r"’bout", "about", sentence)

    sentence = re.sub(r"'til", "until", sentence)
    sentence = re.sub(r"’til", "until", sentence)

    sentence = re.sub(r"c'mon", "come on", sentence)
    sentence = re.sub(r"c’mon", "come on", sentence)
    
    sentence = re.sub("\n", "", sentence)

    sentence = re.sub("[-*/()\"’'#/@;:<>{}`+=~|.!?,]", "", sentence)
    
    return sentence

In [10]:
lines.lines = lines.lines.apply(lambda line: clean_text(line))

In [11]:
lines.head()

,lines
1,in the beginning god created the heaven and th...
2,and the earth was without form and void and da...
3,and god said let there be light and there was ...
4,and god saw the light that it was good and god...
5,and god called the light day and the darkness ...


In [12]:
lines.shape

(31102, 1)

In [13]:
lines.lines = lines.lines.apply(lambda line: line.split())

In [14]:
x_train = [line[:-1] for line in lines.lines]
y_train = [line[1:] for line in lines.lines]

In [15]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [16]:
tokenizer = Tokenizer()

In [17]:
tokenizer.fit_on_texts(lines.lines)

In [18]:
x_train = tokenizer.texts_to_sequences(x_train)
y_train = tokenizer.texts_to_sequences(y_train)

In [19]:
word2idx = tokenizer.word_index
idx2word = {value: key for key, value in word2idx.items()}

In [20]:
word2idx["<pad>"] = 0
idx2word[0] = "<pad>"

In [21]:
lengths = []

for sequence in x_train:
    lengths.append(len(sequence))
    
lengths = pd.Series(lengths)
lengths.describe()

count    31102.000000
mean        24.387917
std         11.079717
min          1.000000
25%         16.000000
50%         23.000000
75%         31.000000
max         89.000000
dtype: float64

In [30]:
maxlen = 90
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 512
vocab_size

12698

In [23]:
x_train = pad_sequences(x_train, maxlen=maxlen, padding='post', truncating='post')
y_train = pad_sequences(y_train, maxlen=maxlen, padding='post', truncating='post')

In [31]:
x_train

array([[   6,    1,  678, ...,    0,    0,    0],
       [   2,    1,  110, ...,    0,    0,    0],
       [   2,   27,   31, ...,    0,    0,    0],
       ...,
       [   2,   77,  121, ...,    0,    0,    0],
       [   7,   28, 5248, ...,    0,    0,    0],
       [   1,  478,    3, ...,    0,    0,    0]])

In [25]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import GRU, Dense, Input, Embedding, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [32]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, mask_zero=True))
model.add(GRU(units=512, return_sequences=True))
model.add(GRU(units=512, return_sequences=True))
model.add(GRU(units=512, return_sequences=True))
model.add(Dense(vocab_size))

In [33]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 512)         6501376   
_________________________________________________________________
gru_3 (GRU)                  (None, None, 512)         1575936   
_________________________________________________________________
gru_4 (GRU)                  (None, None, 512)         1575936   
_________________________________________________________________
gru_5 (GRU)                  (None, None, 512)         1575936   
_________________________________________________________________
dense_1 (Dense)              (None, None, 12698)       6514074   
Total params: 17,743,258
Trainable params: 17,743,258
Non-trainable params: 0
_________________________________________________________________


In [34]:
model.compile(optimizer=Adam(), loss=SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [35]:
history = model.fit(x_train, y_train, epochs=5, verbose=1) #400

Epoch 1/5
972/972 [==============================] - 124s 128ms/step - loss: 1.6867 - accuracy: 0.0821
Epoch 2/5
972/972 [==============================] - 128s 132ms/step - loss: 1.4633 - accuracy: 0.1517
Epoch 3/5
972/972 [==============================] - 127s 131ms/step - loss: 1.2091 - accuracy: 0.2274
Epoch 4/5
972/972 [==============================] - 125s 128ms/step - loss: 1.0985 - accuracy: 0.2599
Epoch 5/5
972/972 [==============================] - 125s 128ms/step - loss: 1.0158 - accuracy: 0.2853


In [ ]:
model.save("model.h5")
#model = load_model("model.h5")

In [43]:
history2 = model.fit(x_train, y_train, epochs=5, verbose=1)

Epoch 1/5
972/972 [==============================] - 127s 130ms/step - loss: 0.9455 - accuracy: 0.3084
Epoch 2/5
972/972 [==============================] - 126s 129ms/step - loss: 0.8824 - accuracy: 0.3337
Epoch 3/5
972/972 [==============================] - 127s 130ms/step - loss: 0.8266 - accuracy: 0.3600
Epoch 4/5
972/972 [==============================] - 126s 130ms/step - loss: 0.7756 - accuracy: 0.3867
Epoch 5/5
972/972 [==============================] - 127s 131ms/step - loss: 0.7300 - accuracy: 0.4121


In [44]:
model.save("model_10_epochs.h5")
#model = load_model("model.h5")

In [45]:
history5 = model.fit(x_train, y_train, epochs=5, verbose=1)

Epoch 1/5
972/972 [==============================] - 127s 130ms/step - loss: 0.6881 - accuracy: 0.4377
Epoch 2/5
972/972 [==============================] - 127s 131ms/step - loss: 0.6500 - accuracy: 0.4619
Epoch 3/5
972/972 [==============================] - 125s 129ms/step - loss: 0.6143 - accuracy: 0.4853
Epoch 4/5
972/972 [==============================] - 125s 129ms/step - loss: 0.5820 - accuracy: 0.5077
Epoch 5/5
972/972 [==============================] - 125s 129ms/step - loss: 0.5524 - accuracy: 0.5281


In [46]:
model.save("model_15_epochs.h5")
#model = load_model("model5.h5")

In [51]:
history7 = model.fit(x_train, y_train, epochs=5, verbose=1)

Epoch 1/5
972/972 [==============================] - 125s 128ms/step - loss: 0.5253 - accuracy: 0.5477
Epoch 2/5
972/972 [==============================] - 125s 129ms/step - loss: 0.4999 - accuracy: 0.5662
Epoch 3/5
972/972 [==============================] - 125s 129ms/step - loss: 0.4772 - accuracy: 0.5840
Epoch 4/5
972/972 [==============================] - 125s 129ms/step - loss: 0.4571 - accuracy: 0.5983
Epoch 5/5
972/972 [==============================] - 125s 129ms/step - loss: 0.4379 - accuracy: 0.6130


In [ ]:
model.save("model7.h5")
#model = load_model("model.h5")

In [55]:
history11 = model.fit(x_train, y_train, epochs=5, verbose=1)

Epoch 1/5
972/972 [==============================] - 125s 128ms/step - loss: 0.4213 - accuracy: 0.6254
Epoch 2/5
972/972 [==============================] - 125s 128ms/step - loss: 0.4055 - accuracy: 0.6377
Epoch 3/5
972/972 [==============================] - 125s 128ms/step - loss: 0.3921 - accuracy: 0.6475
Epoch 4/5
972/972 [==============================] - 125s 128ms/step - loss: 0.3798 - accuracy: 0.6570
Epoch 5/5
972/972 [==============================] - 125s 129ms/step - loss: 0.3683 - accuracy: 0.6655


In [60]:
history12 = model.fit(x_train, y_train, epochs=5)

Epoch 1/5
972/972 [==============================] - 125s 129ms/step - loss: 0.3581 - accuracy: 0.6739
Epoch 2/5
972/972 [==============================] - 126s 130ms/step - loss: 0.3491 - accuracy: 0.6801
Epoch 3/5
972/972 [==============================] - 125s 129ms/step - loss: 0.3413 - accuracy: 0.6855
Epoch 4/5
972/972 [==============================] - 126s 129ms/step - loss: 0.3339 - accuracy: 0.6914
Epoch 5/5
972/972 [==============================] - 126s 129ms/step - loss: 0.3277 - accuracy: 0.6956


In [100]:
model.save('model_30_epochs.h5')

In [93]:
test_text = ["go and sin no more"]

In [94]:
test_sequence = tokenizer.texts_to_sequences(test_text)

In [95]:
test_padded = np.array(pad_sequences(test_sequence, maxlen=maxlen, padding='post', truncating='post'))

In [96]:
test_padded

array([[ 80,   2, 215,  86, 152,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]])

In [97]:
result = [' '.join([idx2word[np.argmax(arr)] for arr in model.predict(test_padded)[0]])]

In [98]:
result_sequence = tokenizer.texts_to_sequences(result)

In [99]:
print(' '.join([idx2word[np.argmax(arr)] for arr in model.predict(result_sequence)[0]]))

the kindly unto unto the not not not of not of not of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of


In [36]:
def generate(word):
    word = clean_text(word)
    inputs = np.zeros((1, 1))
    inputs[0, 0] = word2idx[word]
    count = 1
    while count <= 100:
        pred = model.predict(inputs)
        word = np.argmax(pred)
        if word >= vocab_size:
            word = vocab_size - 1

        inputs[0, 0] = word
        
        print(idx2word[word], end=" ")
        count += 1


In [65]:
generate("glory")

to him that the lord i will he that the lord i will he that the lord i will he that the lord i will he that the lord i will he that the lord i will he that the lord i will he that the lord i will he that the lord i will he that the lord i will he that the lord i will he that the lord i will he that the lord i will he that the lord i will he that the lord i will he that the lord i will he that the 

In [62]:
generate("jesus")

answered abraham begat glory to him that the lord i will he that the lord i will he that the lord i will he that the lord i will he that the lord i will he that the lord i will he that the lord i will he that the lord i will he that the lord i will he that the lord i will he that the lord i will he that the lord i will he that the lord i will he that the lord i will he that the lord i will he that the lord i 

In [63]:
generate("sin")

of the lord i will he that the lord i will he that the lord i will he that the lord i will he that the lord i will he that the lord i will he that the lord i will he that the lord i will he that the lord i will he that the lord i will he that the lord i will he that the lord i will he that the lord i will he that the lord i will he that the lord i will he that the lord i will he that the lord i 

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(50), history.history['loss'])